In [1]:
import json
from collections import Counter


class Article():
    def __init__(self,text):
        try:
            bulk = text.split('#')
            if len(bulk) != 6:
                self.kill = True
            else:
                self.kill = False
                self.id = bulk[0]
                self.title = bulk[1]
                self.info_box_type = bulk[2]
                self.clean_info_box = self.map_json_keys(json.loads(bulk[3].replace('karakteradı','ad').replace('adı','ad')))
                self.bulk_info_box = json.loads(bulk[4])
                self.paragraph = bulk[5]
                self.filed_counter = Counter(self.clean_info_box)
        except Exception as e:
            print e
            #print e
    def map_json_keys(self,d):
        # ad
        for_name =  ['adı','isim','ismi','adi','name','karakteradı']
        for n in for_name:
            try : 
                d['ad']= json.dumps(d[n], ensure_ascii=False, encoding='utf8')
                del d[n]
            except Exception as e:
                continue
        return d
        '''
        
        # doğumyeri
        for_birth_place = ['doğum_yeri']
        # 'resim' varsa sil
        '''
        
def reader(articles_path):
    article_text = open(articles_path, "r") .read()
    articles = []
    for a in article_text.split('\n\n\n'):
        article = Article(a)
        if article.kill == False:
            articles.append(article)
    return articles


In [2]:
article_path ='/Users/uluc/Desktop/Bitirme/Wikiparse_WorkSpace/<2018.10.-->Wiki/<2018-10-21>Outputs-ArticleClean/Clean_Interested_Articles.txt'
articles = reader(article_path)



In [3]:
counter_of_fields = {}

for a in articles:
    try:
        counter_of_fields[a.info_box_type]['count'] += Counter(a.clean_info_box.keys())
        counter_of_fields[a.info_box_type]['number'] += 1 
    except Exception as e:
        counter_of_fields[a.info_box_type] = {}
        counter_of_fields[a.info_box_type]['count'] = Counter(a.clean_info_box.keys())
        counter_of_fields[a.info_box_type]['number'] = 1 


In [4]:
from datetime import date
import os
def create_files():
    today = date.today().strftime('<%Y-%m-%d>')
    mypath = '../'+today+'Outputs-DataFieldCount'
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    log_path = '../'+today+'Outputs-DataFieldCount/DataFieldCount_Report.txt'
    f= open(log_path,"w")
    
    ex_path = '../'+today+'Outputs-DataFieldCount/Counters'
    if not os.path.isdir(ex_path):
        os.makedirs(ex_path)
    uniq_path = '../'+today+'Outputs-DataFieldCount/Uniq'
    if not os.path.isdir(uniq_path):
        os.makedirs(uniq_path)
    return mypath+'/',ex_path+'/',log_path,uniq_path+'/'


In [5]:
output_path, ex_path, log_path,uniq_path = create_files()

In [6]:
# tüm countları bastırma
allCounters = Counter()
number = 0
for i in counter_of_fields.keys():
    allCounters +=counter_of_fields[i]['count']
    number += counter_of_fields[i]['number']

f = open(output_path+'all_count.txt',"w")
aa = '#Info Box Type : {}, #Total article with infoBox: {}\n\n'.format(len(counter_of_fields.keys()),number)
a = json.dumps(allCounters.most_common(), ensure_ascii=False, encoding='utf8',indent = 4).encode('utf-8')
f.write(aa)
f.write(a)
f.close()

In [7]:
# her bir cat için bastırma counter
for i in counter_of_fields.keys():
    f = open(ex_path+str(i)+'.txt',"w")
    aa = 'Info Box Type : {}, #Total article with this infoBox: {}\n\n'.format(i,counter_of_fields[i]['number'])
    a = json.dumps(counter_of_fields[i]['count'].most_common(), ensure_ascii=False, encoding='utf8',indent = 4).encode('utf-8')
    f.write(aa)
    f.write(a)
    f.close()


In [8]:
def give_uniq_fields(all_types,who,uniq_path):
    st = 'Uniq Data Fields for {} -------\n'.format(all_types.keys()[who])
    one_type = all_types[all_types.keys()[who]]['count']
    other_types = {}
    other_types =  [ all_types[k]['count']  for k in all_types.keys() if k != all_types.keys()[who]]
    result = []
    for element in one_type.keys():
        Flag = False
        for cs in other_types:
            if cs[element] != 0:
                Flag=True
        if Flag == False:
            result.append(element)
    #strr = ' \n'.join(result)
    f= open(uniq_path+all_types.keys()[who]+'_Uniq_Fields.txt',"w")
    f.write(st)
    f.write(json.dumps(result, ensure_ascii=False, encoding='utf8',indent = 4).encode('utf-8'))
    
    
for ii in range (0,len(counter_of_fields.keys())):
    give_uniq_fields (counter_of_fields,ii,uniq_path)

In [64]:
# çıktı göresterme
f= open(output_path+'Counts2.txt',"w")
f.close()
for i in counter_of_fields.keys():
    f= open(output_path+'Counts2.txt',"ab")
    a = []
    for j in counter_of_fields[i]['count'].most_common(10):
        y = counter_of_fields[i]['number']
        b = json.dumps(j, ensure_ascii=False, encoding='utf8').\
                    replace(']','').replace('[','')\
                    .replace('"','').replace(',',':').strip()
        bb = int(b.split(':')[1].strip())
        a.append( ('%26s|')%( b ))
    u = unicode(i, "utf-8")
    ab = u.rjust(22,' ')+str(counter_of_fields[i]['number']).rjust(6,' ')+' ->'
    ab = ab.encode('utf8')
    for bb in a:
        ab += bb.encode('utf-8')
    f.write(ab)
    f.write('\n') 
    f.close()
    




In [65]:
# çıktı göresterme
f= open(output_path+'Counts2.txt',"ab")
f.write('\n')
for i in counter_of_fields.keys():
    a = []
    for j in counter_of_fields[i]['count'].most_common(10):
        y = counter_of_fields[i]['number']
        b = json.dumps(j, ensure_ascii=False, encoding='utf8').\
                    replace(']','').replace('[','')\
                    .replace('"','').replace(',',':').strip()
        bb = int(b.split(':')[1].strip())
        a.append( ('%20s: %.2f|')%( b.split(':')[0].strip(),bb/float(y) ))
    u = unicode(i, "utf-8")
    ab = u.rjust(22,' ')+str(counter_of_fields[i]['number']).rjust(6,' ')+' ->'
    ab = ab.encode('utf8')
    for bb in a:
        ab += bb.encode('utf-8')
    f.write(ab)
    f.write('\n')
f.close()
    




In [66]:
# çıktı göresterme
f= open(output_path+'Counts2.txt',"ab")
f.write('\n')
for i in counter_of_fields.keys():
    a = []
    for j in counter_of_fields[i]['count'].most_common(10):
        y = counter_of_fields[i]['number']
        b = json.dumps(j, ensure_ascii=False, encoding='utf8').\
                    replace(']','').replace('[','')\
                    .replace('"','').replace(',',':').strip()
        bb = int(b.split(':')[1].strip())
        if bb/float(y)>=0.60:
            a.append( ('%20s: %.2f|')%( b.split(':')[0].strip(),bb/float(y) ))
        else:
            a.append( ('%26s|')%( '' ))
    u = unicode(i, "utf-8")
    ab = u.rjust(22,' ')+str(counter_of_fields[i]['number']).rjust(6,' ')+' ->'
    ab = ab.encode('utf8')
    for bb in a:
        ab += bb.encode('utf-8')
    f.write(ab)
    f.write('\n')
f.close()
    




In [67]:
f = open(output_path+'Counts2.txt',"ab")    
f.write('\n\nGenel durum\n')    
total = ('%25s|')%(json.dumps(allCounters.most_common(20), ensure_ascii=False, encoding='utf8').\
                    replace(']','').replace('[','')\
                    .replace('"','').replace(',',':').strip() )
f.write(total.encode('utf-8'))
f.close()


In [43]:
def save_clean_articles(output_path,articles):
    fd= open(output_path+'All_Article.txt',"w")
    fd.close()
    f= open(output_path+'All_Article.txt',"ab")
    for a in articles:
        try:
            bd = json.dumps(a.clean_info_box, ensure_ascii=False, encoding='utf8').encode('utf-8')
            cd = json.dumps(a.bulk_info_box, ensure_ascii=False, encoding='utf8').encode('utf-8')
        except Exception as e:
            continue
        f.write(str(a.id) +'#')
        f.write(str(a.title) +'#')
        f.write(str(a.info_box_type) +'#')
        f.write(bd)
        f.write('#')
        f.write(cd)
        f.write('#')
        f.write(str(a.paragraph)) 
        f.write('\n\n\n')
    f.close()


In [39]:
save_clean_articles(output_path,articles)